In [8]:
import os
import PIL 
from PIL import Image 

images = os.listdir()

# get pixel size of images
for image in images:
    # # To restart gpt4o detection
    # if image.endswith('.json'):
    #     os.remove(image)
    #     continue

    if not image.endswith('.jpg'):
        continue

    # For simplicity, let's ignore images with "-" in the name
    if "-" in image:
        os.remove(image)
        continue

    # loading the image 
    img = PIL.Image.open(os.path.join(image)) 
    
    # fetching the dimensions 
    wid, hgt = img.size 
    
    # displaying the dimensions 
    print(str(wid) + "x" + str(hgt))

768x1020
750x1000
750x1000
1000x750
972x1296
768x1020
750x1000
563x1000
768x1020
780x1040
1000x750
780x1040
750x1000
1020x768
972x1296
780x1040
1224x1632
1000x750
648x486
1000x750
768x1020
768x1020
750x1000
1224x1632
780x1040
1020x768
648x486
816x612
780x1040
750x1000
972x1296
750x1000
1000x750
780x1040
780x1040
648x486
780x1040
750x1000
1000x750
972x1296
1224x1632
1040x780
780x1040
768x1020
750x1000
780x1040
1020x768
780x1040
1000x750
1224x1632
1224x1632
1040x780
756x1008
1020x768
756x1008
1020x768
780x1040
750x1000
750x1000
780x1040
972x1296
750x1000
1008x756
780x1040
750x1000
780x1040
816x612
1000x750
756x1008
756x1008
768x1020
1000x750
780x1040
1224x1632
1040x780
780x1040
563x1000
1000x750
780x1040
768x1020
780x1040
768x1020
1224x1632
780x1040
780x1040
1224x1632
750x1000
1224x1632
1000x750
972x1296
563x1000
1224x1632
1000x750
563x1000
756x1008
750x1000
1224x1632
972x1296
780x1040
780x1040


### Getting JSON Object for all 300 Images

In [9]:
from openai import OpenAI
import base64
import json
from tqdm import tqdm

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

api_key = "sk-proj-b5fNMH4ImSHeSuH0LZHBT3BlbkFJlAJYuvUIDBIWM5edcmI0"
client = OpenAI(api_key=api_key)
prompt = """From the given prescription image, what's the name of the institution name, doctor name, the prescribed medication, and the quantity of each medication?
Note: if prescription does not have a doctor name, provide the name of the institution instead, with the key "institution" instead of "doctor" in the JSON response.
If the prescription is not clear, please return an empty JSON object.
Respond with JSON format. 

For example:
{
  "doctor": "Doctor Name",
  "medication": [
    {
      "name": "Medicine Name",
      "quantity": "Medicine Quantity"
    },
    {
        "name": "Medicine Name",
        "quantity": "Medicine Quantity"
    }
  ],
}
"""
for image_path in tqdm(images):
    # if not a jpg file, skip
    if not image_path.endswith('.jpg'):
        continue
    
    # If corresponding JSON file exists, skip
    save_file = f"{image_path.split('.')[0]}.json"
    if os.path.exists(save_file):
        continue

    
    # Getting the base64 string
    base64_image = encode_image(image_path)

    while True:
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {   "type": "image_url", 
                                "image_url": {
                                    "url": f"data:image/jpeg;base64,{base64_image}",
                                    "detail": "low"  # treats the image as 512x512 pixels. Equivalent to 85 tokens
                                }},
                            {
                                "type": "text",
                                "text": prompt,
                            },
                        ],
                    }
                ],
                temperature=0,
                max_tokens=512,
                response_format={ "type": "json_object" }
            )

            with open(save_file, 'w') as f:
                obj = json.loads(response.choices[0].message.content)
                json.dump(obj, f, indent=4)
            
            break
        except Exception as e:
            print(e)
            print("Retrying...")

 99%|█████████▉| 102/103 [06:08<00:03,  3.46s/it]

Unterminated string starting at: line 86 column 19 (char 1348)
Retrying...


100%|██████████| 103/103 [06:20<00:00,  3.70s/it]
